In [1]:
import webbrowser
from http.server import HTTPServer, BaseHTTPRequestHandler
import threading
from urllib.parse import urlparse, parse_qs


class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        # Parse the URL
        parsed_path = urlparse(self.path)
        # Extract query parameters as a dictionary
        query_params = parse_qs(parsed_path.query)

        # Assuming the token is passed as a query parameter named 'token'
        global auth_token
        auth_token = query_params.get("token", [None])[0]

        # Send a response to the browser
        self.send_response(200)
        self.end_headers()
        message = "Authentication successful. You can close this tab."
        self.wfile.write(message.encode("utf-8"))

        # Print the token or handle it as needed
        print("Received token:", auth_token)

        # Signal the server to stop after handling the request
        global server_should_stop
        server_should_stop = True


# Specify the port to listen on. Choose a port that's likely to be free.
port = 8000

# Create the server object
httpd = HTTPServer(("localhost", port), SimpleHTTPRequestHandler)


# Function to start the server and listen until a stop signal is received
def run_server():
    global server_should_stop
    server_should_stop = False
    while not server_should_stop:
        httpd.handle_request()
    httpd.server_close()


# Start the server in a new thread because handle_request() is blocking
server_thread = threading.Thread(target=run_server)
server_thread.start()

# Direct the user to the Snowflake login page using the provided account URL
# Replace YOUR_CLIENT_ID with your actual client ID
# Ensure the redirect_uri matches one of the URIs you've registered in your Snowflake OAuth integration
login_url = "https://bu20658.ap-southeast-2.snowflakecomputing.com/?&client_id=KEaAv5tJAYc1%2BBqySu5DRtAL1DQ%2Frw%3D%3D&scope=refresh_token&redirect_uri=http://localhost:8000"
webbrowser.open(login_url)


# The script will pause here and wait for the server thread to finish
server_thread.join()

# At this point, `auth_token` should contain the OAuth token if the authentication was successful
print("Auth Token:", auth_token)

Received token: None
Auth Token: None


127.0.0.1 - - [08/Apr/2024 11:15:10] "GET / HTTP/1.1" 200 -
